# Train

In [1]:
import os
import os.path as osp

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

from model import FaceMobileNet
from model.metric import ArcFace, CosFace
from model.loss import FocalLoss
from dataset import load_data
from config import Config as conf

import numpy as np
from PIL import Image

In [2]:
from dataset import load_data

In [3]:
load_data(conf)

NameError: name 'np' is not defined

In [2]:
os.chdir("D:\Course\Graduate1\Data_Science\hw\hw5")

In [3]:
# Data Setup
dataloader, class_num = load_data(conf, training=True)
embedding_size = conf.embedding_size
device = conf.device

In [4]:
# Network Setup
net = FaceMobileNet(embedding_size).to(device)

if conf.metric == 'arcface':
    metric = ArcFace(embedding_size, class_num).to(device)
else:
    metric = CosFace(embedding_size, class_num).to(device)

net = nn.DataParallel(net)
metric = nn.DataParallel(metric)

In [5]:
# Training Setup
if conf.loss == 'focal_loss':
    criterion = FocalLoss(gamma=2)
else:
    criterion = nn.CrossEntropyLoss()

if conf.optimizer == 'sgd':
    optimizer = optim.SGD([{'params': net.parameters()}, {'params': metric.parameters()}], 
                            lr=conf.lr, weight_decay=conf.weight_decay)
else:
    optimizer = optim.Adam([{'params': net.parameters()}, {'params': metric.parameters()}],
                            lr=conf.lr, weight_decay=conf.weight_decay)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=conf.lr_step, gamma=0.1)

In [6]:
# Checkpoints Setup
os.makedirs(conf.checkpoints, exist_ok=True)

In [12]:
# Start training
net.train()

for e in range(conf.epoch):
    for data, labels in tqdm(dataloader, desc=f"Epoch {e}/{conf.epoch}",
                             ascii=True, total=len(dataloader)):
        data = data.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        embeddings = net(data)
        thetas = metric(embeddings, labels)
        loss = criterion(thetas, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {e}/{conf.epoch}, Loss: {loss}")

    backbone_path = osp.join(conf.checkpoints, f"{e}.pth")
    torch.save(net.state_dict(), backbone_path)
    scheduler.step()

Epoch 0/30: 100%|##########| 7119/7119 [14:31<00:00,  8.17it/s]


Epoch 0/30, Loss: 14.636209487915039


Epoch 1/30: 100%|##########| 7119/7119 [14:30<00:00,  8.18it/s]


Epoch 1/30, Loss: 13.404420852661133


Epoch 2/30: 100%|##########| 7119/7119 [14:30<00:00,  8.18it/s]


...

Epoch 27/30, Loss: 5.544327735900879


Epoch 28/30: 100%|##########| 7119/7119 [15:04<00:00,  7.87it/s]


Epoch 28/30, Loss: 5.775236129760742


Epoch 29/30: 100%|##########| 7119/7119 [14:53<00:00,  7.96it/s]

Epoch 29/30, Loss: 6.29815149307251


# Test

In [3]:
def unique_image(pair_list) -> set:
    """Return unique image path in pair_list.txt"""
    with open(pair_list, 'r') as fd:
        pairs = fd.readlines()
    unique = set()
    for pair in pairs:
        id1, id2, _ = pair.split()
        unique.add(id1)
        unique.add(id2)
    return unique

In [4]:
def group_image(images: set, batch) -> list:
    """Group image paths by batch size"""
    images = list(images)
    size = len(images)
    res = []
    for i in range(0, size, batch):
        end = min(batch + i, size)
        res.append(images[i : end])
    return res

In [5]:
def _preprocess(images: list, transform, low_light = False) -> torch.Tensor:
    res = []
    for img in images:
        im = Image.open(img)
        if low_light:
            im = (np.sqrt (im)*2).astype(np.uint8)
            im = Image.fromarray(im)
        im = transform(im)
        res.append(im)
    data = torch.cat(res, dim=0)  # shape: (batch, 128, 128)
    data = data[:, None, :, :]    # shape: (batch, 1, 128, 128)
    return data

In [6]:
def featurize(images: list, transform, net, device,low_light = False) -> dict:
    """featurize each image and save into a dictionary
    Args:
        images: image paths
        transform: test transform
        net: pretrained model
        device: cpu or cuda
    Returns:
        Dict (key: imagePath, value: feature)
    """
    data = _preprocess(images, transform,low_light = low_light)
    data = data.to(device)
    net = net.to(device)
    with torch.no_grad():
        features = net(data) 
    res = {img: feature for (img, feature) in zip(images, features)}
    return res

In [7]:
def cosin_metric(x1, x2):
    return np.dot(x1, x2) / (np.linalg.norm(x1) * np.linalg.norm(x2))

In [8]:
def threshold_search(y_score, y_true):
    y_score = np.asarray(y_score)
    y_true = np.asarray(y_true)
    best_acc = 0
    best_th = 0
    for i in range(len(y_score)):
        th = y_score[i]
        y_test = (y_score >= th)
        acc = np.mean((y_test == y_true).astype(int))
        if acc > best_acc:
            best_acc = acc
            best_th = th
    return best_acc, best_th

In [9]:
def compute_accuracy(feature_dict, pair_list, test_root):
    with open(pair_list, 'r') as f:
        pairs = f.readlines()

    similarities = []
    labels = []
    for pair in pairs:
        img1, img2, label = pair.split()
        img1 = osp.join(test_root, img1)
        img2 = osp.join(test_root, img2)
        feature1 = feature_dict[img1].cpu().numpy()
        feature2 = feature_dict[img2].cpu().numpy()
        label = int(label)

        similarity = cosin_metric(feature1, feature2)
        similarities.append(similarity)
        labels.append(label)

    accuracy, threshold = threshold_search(similarities, labels)
    return accuracy, threshold

In [11]:
model = FaceMobileNet(conf.embedding_size)
model = nn.DataParallel(model)
model.load_state_dict(torch.load(conf.test_model, map_location=conf.device))
model.eval()

images = unique_image(conf.test_list)
images = [osp.join(conf.test_root, img) for img in images]
groups = group_image(images, conf.test_batch_size)

feature_dict = dict()
for group in groups:
    d = featurize(group, conf.test_transform, model, conf.device,low_light = False)
    feature_dict.update(d) 
accuracy, threshold = compute_accuracy(feature_dict, conf.test_list, conf.test_root) 

print(
    f"Test Model: {conf.test_model}\n"
    f"Accuracy: {accuracy:.3f}\n"
    f"Threshold: {threshold:.3f}\n"
)

Test Model: checkpoints/26.pth
Accuracy: 0.945
Threshold: 0.308

